# Genome length data across all viruses with known hosts


Log10 genome lengths (x axis), Normalized frequency (y axis)
Plots of genome length vs. number of genes 

In [2]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import ete3 as ete
import os
import Bio
import random
import scipy.stats as stats
from functools import *
from Bio import Entrez
from Bio import SeqIO
import matplotlib.patches as mpatches
import seaborn as sns
sns.set_context("paper")

#filename that will be the name of both the stat text file and the plot. 
filename = 'genomeLengthHistograms'
inputFileNameTXT = filename + '.txt'
fo = open(inputFileNameTXT, 'w')
#writing the first line here, and the rest as part of the for loop 
fo.write('\t'+'Genome Length Statistics For Different Viral Groups (no. of bases)'+'\n')
fo.write('\t'+ 'Min' +'\t'+ 'Max' + '\t'+ '25th Percentile'+'\t'+ 'Median'+ '\t' + '75th Percentile' +'\t' + 'Mean' + '\t' +'Stdev.' +'\t'+ 'Min (Log10)' +'\t'+ 'Max (Log10)' + '\t'+ '25th percentile (Log10)'+'\t'+ 'Median (Log10)'+ '\t' + '75th percentile (Log10)' +'\t' + 'Mean (Log10)' + '\t' +'Stdev. (Log10)' +'\n')

#as part of quality control the following viruses had to be omitted from our analysis
#they contain various types of misannotations or mislabeled hosts 
badFolders = ['Acyrthosiphon_pisum_secondary_endosymbiont_phage_1_uid14047', 'Aureococcus_anophagefferens_virus_MM_2014_uid258005', 'Dragonfly_associated_microphage_1_uid177547', 'Enterobacter_phage_PG7_uid240014', 'Escherichia_phage_vB_EcoM_JS09_uid248321', 'Lactobacillus_phage_LL_H_uid19803', 'Providencia_phage_Redjac_uid177540', 'Pseudoalteromonas_phage_RIO_1_uid206039', 'Sputnik_virophage_2_uid243067', 'Sputnik_virophage_3_uid243065', 'Staphylococcus_phage_6ec_uid253318', 'Streptococcus_phage_DCC1738_uid253219', 'Streptococcus_phage_IC1_uid253220', 'Streptococcus_phage_K13_uid253223', 'Streptococcus_phage_SMP_uid18529', 'Vibrio_phage_CP_T1_uid181062', 'Vibrio_phage_vpms1_uid212709', 'Vibrio_phage_VPUSM_8_uid227006',  'Abaca_bunchy_top_virus_uid28697','Shallot_yellow_stripe_virus_uid15745','Equine_rhinitis_A_virus_uid15205','African_pouched_rat_arterivirus_uid274797','Spilanthes_yellow_vein_virus_uid19779', 'Velvet_bean_severe_mosaic_virus_uid41175','Paramecium_bursaria_Chlorella_virus_FR483_uid18305','Human_endogenous_retrovirus_K113_uid222261','Acholeplasma_phage_MV_L1_uid14573']
#Another group of vviruses with misannotations, this time in their taxonomy (they don't belong to any Baltimore group)
unknowns= ['Bacillus_phage_phBC6A51_uid15021', 'Bacillus_phage_phBC6A52_uid15022', 'Badger_feces_associated_gemycircularvirus_uid281125', 'Chaetoceros_DNA_virus_7_uid237558', 'Chaetoceros_lorenzianus_DNA_Virus_uid63565', 'Chaetoceros_tenuissimus_DNA_virus_uid60753', 'Cladosporium_cladosporioides_virus_1_uid258308', 'Enterobacteria_phage_YYZ_2008_uid32231', 'European_mountain_ash_ringspot_associated_virus_uid39973', 'Faecal_associated_gemycircularvirus_3_uid268557', 'Faecal_associated_gemycircularvirus_4_uid268556', 'Faecal_associated_gemycircularvirus_5_uid268555', 'Faecal_associated_gemycircularvirus_6_uid268554', 'Faecal_associated_gemycircularvirus_7_uid268553', 'Gentian_ovary_ring_spot_virus_uid256090', 'Geobacillus_virus_E2_uid19797', 'Glossina_pallidipes_salivary_gland_hypertrophy_virus_uid28839', 'Haloarcula_hispanica_icosahedral_virus_2_uid109269', 'Haloarcula_phage_SH1_uid15535', 'Halovirus_PH1_uid196975', 'Halovirus_VNH_1_uid262927', 'Human_genital_associated_circular_DNA_virus_1_uid281399', 'Lactobacillus_johnsonii_prophage_Lj771_uid28145', 'Magnaporthe_oryzae_RNA_virus_uid272442', 'Mongoose_feces_associated_gemycircularvirus_a_uid281407', 'Mongoose_feces_associated_gemycircularvirus_b_uid281406', 'Mongoose_feces_associated_gemycircularvirus_c_uid281404', 'Mongoose_feces_associated_gemycircularvirus_d_uid281405', 'Mycoplasma_phage_phiMFV1_uid14387', 'Penicillium_roqueforti_dsRNA_mycovirus_1_uid258162', 'Phage_Gifsy_1_uid32269', 'Phage_Gifsy_2_uid32271', 'Pigeonpea_cryptic_virus_uid244664', 'Planaria_asexual_strain_specific_virus_like_element_type_1_uid14140', 'Pseudomonas_phage_phi_Pto_bp6g_uid240724', 'Rhizoctonia_fumigata_mycovirus_uid283068', 'Rhodococcus_phage_REQ2_uid81171', 'Rhodococcus_phage_REQ3_uid81175', 'Rose_rosette_virus_uid64937', 'Rosellinia_necatrix_fusarivirus_1_uid255787', 'Rosellinia_necatrix_megabirnavirus_1_W779_uid41609', 'Salisaeta_icosahedral_phage_1_uid167575', 'Salmonella_phage_Fels_1_uid29267', 'Sodalis_phage_phiSG1_uid16583', 'Staphylococcus_phage_phi2958PVL_uid32173', 'Staphylococcus_phage_tp310_1_uid20659', 'Staphylococcus_phage_tp310_3_uid20663', 'Stenotrophomonas_phage_phiSMA9_uid15493', 'Streptococcus_phage_20617_uid239271', 'Streptococcus_phage_phiBHN167_uid227353', 'Streptococcus_pyogenes_phage_315_1_uid14533', 'Streptococcus_pyogenes_phage_315_2_uid14528', 'Streptococcus_pyogenes_phage_315_3_uid14529', 'Streptococcus_pyogenes_phage_315_4_uid14530', 'Streptococcus_pyogenes_phage_315_5_uid14531', 'Streptococcus_pyogenes_phage_315_6_uid14532', 'Tanay_virus_uid246129', 'Thermococcus_prieurii_virus_1_uid84407', 'Thermus_phage_IN93_uid14235', 'Thermus_phage_P23_77_uid40235']
badFolders=badFolders+unknowns
#these are satellite viruses, which we will exclude from further analysis in this manuscript 
sats = ['Ageratum_conyzoides_associated_symptomless_alphasatellite_uid259293', 'Ageratum_enation_alphasatellite_uid181994', 'Ageratum_leaf_curl_betasatellite_uid195929', 'Ageratum_leaf_curl_Cameroon_betasatellite_uid36669', 'Ageratum_yellow_leaf_curl_betasatellite_uid14439', 'Ageratum_yellow_vein_China_alphasatellite_uid237561', 'Ageratum_yellow_vein_China_virus_associated_DNA_beta_uid15515', 'Ageratum_yellow_vein_Singapore_alphasatellite_uid14232', 'Ageratum_yellow_vein_virus_satellite_DNA_beta_uid14444', 'Alternanthera_yellow_vein_virus_satellite_DNA_beta_uid19833', 'Andrographis_yellow_vein_leaf_curl_betasatellite_uid243492', 'Bhendi_yellow_vein_India_betasatellite__India_Aurangabad_OY164_2006__uid61557', 'Bhendi_yellow_vein_mosaic_betasatellite__India_Coimbator_OYCO1_2005__uid61777', 'Bhendi_yellow_vein_mosaic_virus_associated_alphasatellite_uid174781', 'Bhendi_yellow_vein_mosaic_virus_satellite_DNA_beta_uid14445', 'Black_medic_leafroll_alphasatellite_1_uid243500', 'Cardamom_bushy_dwarf_virus_satellite_uid230064', 'Cassava_mosaic_Madagascar_alphasatellite_uid175666', 'Chili_leaf_curl_Bhatinda_betasatellite_uid206467', 'Chilli_leaf_curl_alphasatellite_uid253224', 'Chilli_leaf_curl_Multan_alphasatellite_uid39933', 'Chilli_leaf_curl_virus_satellite_DNA_beta_uid14441', 'Cleome_leaf_crumple_virus_associated_DNA_1_uid60045', 'Corchorus_yellow_vein_mosaic_betasatellite_uid192608', 'Cotton_leaf_curl_Burewala_alphasatellite_uid45935', 'Cotton_leaf_curl_Burewala_betasatellite_uid45933', 'Cotton_leaf_curl_Gezira_alphasatellite_uid42507', 'Cotton_leaf_curl_Gezira_virus_satellite_DNA_beta_uid15166', 'Cotton_leaf_curl_Multan_betasatellite_uid15780', 'Cotton_leaf_curl_virus_associated_DNA_1_isolate_Lucknow_uid65305', 'Cotton_leaf_curl_virus_associated_DNA_beta_uid14438', 'Cotton_leaf_curl_virus_betasatellite_uid162497', 'Cowpea_severe_leaf_curl_associated_DNA_beta_uid15157', 'Croton_yellow_vein_mosaic_alphasatellite_uid45931', 'Croton_yellow_vein_mosaic_betasatellite_uid18249', 'Cuban_alphasatellite_1_uid210798', 'Dragonfly_associated_alphasatellite_uid181244', 'Emilia_yellow_vein_virus_associated_DNA_beta_uid37893', 'Erectites_yellow_mosaic_virus_satellite_DNA_beta_uid19827', 'Eupatorium_yellow_vein_virus_satellite_DNA_beta_uid14447', 'Faba_bean_necrotic_stunt_alphasatellite_1_uid243499', 'Faba_bean_necrotic_stunt_alphasatellite_2_uid243498', 'French_bean_leaf_curl_betasatellite_Kanpur_uid169556', 'Gossypium_darwinii_symptomless_alphasatellite_uid39593', 'Gossypium_davidsonii_symptomless_alphasatellite_uid39589', 'Gossypium_mustilinum_symptomless_alphasatellite_uid39591', 'Grapevine_satellite_virus_uid208539', 'Guar_leaf_curl_alphasatellite_uid193981', 'Hedyotis_uncinella_yellow_mosaic_betasatellite_uid230991', 'Honeysuckle_yellow_vein_mosaic_disease_associated_satellite_DNA_beta_uid19863', 'Honeysuckle_yellow_vein_mosaic_virus_satellite_DNA_beta_uid14620', 'Leucas_zeylanica_yellow_vein_virus_satellite_DNA_beta_uid41305', 'Ludwigia_leaf_distortion_betasatellite__India_Amadalavalasa_Hibiscus_2007__uid29233', 'Ludwigia_yellow_vein_virus_associated_DNA_beta_uid15561', 'Luffa_puckering_and_leaf_distortion_associated_betasatellite__India_Gurdaspur_Okra_2013__uid253218', 'Luffa_puckering_and_leaf_distortion_associated_DNA_beta_uid15779', 'Malachra_yellow_vein_mosaic_virus_associated_satellite_DNA_beta_uid28727', 'Malvastrum_leaf_curl_betasatellite_uid16301', 'Malvastrum_leaf_curl_Guangdong_betasatellite_uid243827', 'Malvastrum_yellow_vein_Yunnan_virus_satellite_DNA_beta_uid14567', 'Melon_chlorotic_mosaic_virus_associated_alphasatellite_uid51413', 'Mesta_yellow_vein_mosaic_virus_alphasatellite__India_Hoshiarpur_Okra_2013__uid253217', 'Mesta_yellow_vein_mosaic_virus_associated_alphasatellite_uid174780', 'Mesta_yellow_vein_mosaic_virus_associated_DNA_beta_uid21015', 'Milk_vetch_dwarf_C10_alphasatellite_uid240682', 'Mirabilis_leaf_curl_India_virus_associated_betasatellite_uid255800', 'Mungbean_yellow_mosaic_India_virus_associated_betasatellite__India__Faizabad__Cow_Pea_2012__uid177773', 'Nanovirus_like_particle_uid14386', 'Okra_enation_leaf_curl_betasatellite__India_Sonipat_EL10_2006__uid61781', 'Okra_leaf_curl_Mali_virus_satellite_DNA_beta_uid20323', 'Okra_yellow_vein_disease_associated_sequence_uid14443', 'Olive_viral_satellite_RNA_uid230268', 'Papaya_leaf_curl_alphasatellite_uid233414', 'Papaya_leaf_curl_China_virus_satellite_DNA_beta_uid19819', 'Pepper_leaf_curl_virus_satellite_DNA_beta_uid28283', 'Pepper_leaf_curl_Yunnan_virus_satellite_DNA_beta_uid29415', 'Potato_apical_leaf_curl_disease_associated_satellite_DNA_beta_uid18323', 'Radish_leaf_curl_virus_betasatellite_uid28281', 'Sida_leaf_curl_virus_satellite_DNA_beta_uid19823', 'Sida_yellow_mosaic_virus_China_associated_DNA_beta_uid15514', 'Sida_yellow_vein_China_alphasatellite_uid238950', 'Siegesbeckia_yellow_vein_virus_associated_DNA_beta_uid17269', 'Subterranean_clover_stunt_C6_alphasatellite_uid14180', 'Synedrella_leaf_curl_alphasatellite_uid263034', 'Tobacco_leaf_chlorosis_betasatellite_uid178075', 'Tobacco_leaf_curl_disease_associated_sequence_uid14442', 'Tobacco_leaf_curl_PUSA_alphasatellite_uid56023', 'Tobacco_leaf_curl_virus_associated_DNA_beta_uid45925', 'Tomato_leaf_curl_Bangladesh_betasatellite_uid56017', 'Tomato_leaf_curl_Gandhinagar_betasatellite_uid231683', 'Tomato_leaf_curl_Hajipur_betasatellite_uid175587', 'Tomato_leaf_curl_Joydebpur_virus_satellite_DNA_beta_uid28273', 'Tomato_leaf_curl_Karnataka_alphasatellite_uid181995', 'Tomato_leaf_curl_Pakistan_virus_associated_DNA_1_uid38463', 'Tomato_leaf_curl_Patna_betasatellite_uid36541', 'Tomato_leaf_curl_Togo_betasatellite_Togo_2006__uid60629', 'Tomato_leaf_curl_Yemen_betasatellite_uid177643', 'Tomato_yellow_leaf_curl_China_betasatellite_uid181248', 'Tomato_yellow_leaf_curl_Vietnam_virus_satellite_DNA_beta_uid19829', 'TYLCCNV_Y322__satellite_DNA_beta_uid16338', 'Vernonia_yellow_vein_betasatellite_uid41303', 'Vernonia_yellow_vein_Fujian_virus_alphasatellite_uid72145', 'Vernonia_yellow_vein_Fujian_virus_betasatellite_uid72143', 'Zinnia_leaf_curl_disease_associated_sequence_uid14440', 'Zinnia_leaf_curl_virus_associated_DNA_beta_uid14538']

def getTaxidFromGenomeFolder(genome_folder):
    #Takes an rpt file for a genome and returns the taxid in it.
    if folder not in badFolders and folder not in sats:
        with open(os.path.join('all_rpt', folder, os.listdir(os.path.join('all_rpt', folder))[0])) as rpt_file:
            for line in rpt_file:
                if line.startswith('Taxid: '):
                    return int(line[7:-1])
                

def getHostTaxidFromGenomeFolder(genome_folder):    
    #Takes a folder containing an asn file and returns the taxid of the host of the virus described by that asn file.    
    if folder not in badFolders and folder not in sats:
        with open('folderToHostTaxid.txt', 'r') as taxid2host:
            for line in taxid2host:
                if genome_folder in line:
                    return int(line.split('\t')[1][:-1])
        return 0
    

def getGenomeLengthFromGenomeFolder(genome_folder):
    total = 0
    if folder not in badFolders and folder not in sats:
        for file in os.listdir(os.path.join('all_ptt', folder)):
            with open(os.path.join('all_ptt', folder, file)) as ptt_file:
                line = ptt_file.readline()
                index = line.find('- 1..')
                if index == -1:
                    return -1
                num = line[index + 5 : -1]
                total += int(num)
        return total
    

def getCodingNumberAndOverlapNumberFromGenomeFolder(folder):
    totalCoding = 0
    totalOverlap = 0
    for file in os.listdir(os.path.join('all_ptt', folder)):
        #these folders contain genomes with misannotations (e.g in gene overlap, non-coding DNA, or host information from NCBI)
        if folder not in badFolders and folder not in sats:  
            with open(os.path.join('all_ptt', folder, file)) as ptt_file:
                line = ptt_file.readline()
                index = line.find('- 1..')
                repliconLength = int(line[index + 5 : -1])
                indicesThatOverlap = [0] * repliconLength
                line = ptt_file.readline()
                line = ptt_file.readline()
                
                uniqueIndices = set([])
                for line in ptt_file:
                    line = line.split('\t')[0]
                    [start, end] = [int(x) for x in line.split('..')]
                    #this is put in place to distinguish linear from circular genomes. Here, we're looking for linear genomes
                    if start < end:                    
                        for i in range(start, end):
                            if i in uniqueIndices:
                                indicesThatOverlap[i] = 1
                            else:
                                uniqueIndices.add(i)
                    #here were are looking for circular genomes to be able to properly count noncoding bases
                    elif start > end:
                        for i in range(start, repliconLength):
                            if i in uniqueIndices:
                                indicesThatOverlap[i] = 1
                            else:
                                uniqueIndices.add(i)
                        for i in range(1, end):
                            if i in uniqueIndices:
                                indicesThatOverlap[i] = 1
                            else:
                                uniqueIndices.add(i)
                           
                totalCoding += len(uniqueIndices)
                #totalOverlap += len(bigList) - len(uniqueIndices)
                totalOverlap += sum(indicesThatOverlap)
    return (totalCoding, totalOverlap)


def makePlot(virusGroups, hostNames, plotPositionVert):
    #Makes a plot by plotting the current values of virusGroups and adding a legend using hostNames.    
    #plotPositionVert will let the program know where in the 7x3 subplots to place the figures that are generated. 
    #genomeLengthByGroup is a list of lists containing length of genomes per virus group across different classifications
    genomeLengthByGroup = [[(np.log10(x['genomeLength'])) for x in group] for group in virusGroups]
    #yVec is normalized frequency. Basically 1/N, where N is the number of viruses per viral group. 
    yVec = [len([x['genomeLength'] for x in group]) for group in virusGroups]     
    
    #boxWidth is the width of the box plot
    boxWidth = 0.05 
    #matOfOnes will be a list of lists, containing ones. It has the same structure as genomeLengthByGroup
    matOfOnes=[]
    #matWeight will hold 1/N values. 
    matWeight=[]
    #ylimitVec will hold max histogram bar height, so the plot y limit can be determined based off of it. 
    
    counter = range(len(yVec))
    #per each classification system, we're looping through viral groups, which correspond to j. 
    for j in counter:
        vecOfOnes= np.zeros(yVec[j]) + 1.
        matOfOnes.append(vecOfOnes)
        matWeight.append([k / yVec[j] for k in vecOfOnes])   
        
        #we want to make sure that the nucleotide classification plots end up at the 15th and 18th subplot positions
        if plotPositionVert is 4:
            if j is 0:
                plotPositionHor = 14
            elif j is 1: 
                plotPositionHor = 17
            else:
                plotPositionHor = 20
            ax1 = plt.subplot(7,3, plotPositionHor)
        
        #all other subplots are placed automatically according to their plotPositionVert, which is specified later in the code
        else:
            plotPositionHor = 3*j + plotPositionVert
            ax1 = plt.subplot(7,3, plotPositionHor)
        
        #calculating the number of bins needed according to Freedman-Diaconis rule. Bin width = 2 * IQR * N^(-1/3), where N is number of datapoints, IQR is interquartile range. 
        #number of bins can be calculated from bin width according to bin number = (max(data) - min(data))/bin width
        iqr=np.percentile(genomeLengthByGroup[j], 75, interpolation='higher') - np.percentile(genomeLengthByGroup[j], 25, interpolation='lower')
        bottom = binWidth=2*iqr*len(genomeLengthByGroup[j])**(-1/3)
        top = max(genomeLengthByGroup[j])-min(genomeLengthByGroup[j])
        binNumFreed =int(np.ceil(top/bottom))
        
        #plotting the normalized histograms, and ensuring bins of equal length by setting 'bins' equal to an array with equal spacings
        n, bins, patches = plt.hist(genomeLengthByGroup[j], weights=matWeight[j], bins=binNumFreed, color=colors[j], linewidth=0)
        maxHeight= np.round(np.amax(n),1) + 0.2
        
        #instead of a legend, since there is only one figure per plot, I'm annotating the plot with virus group name, and number of viruses per that group 
        ax1.annotate(hostNames[j] + ' (N = ' + str(yVec[j]) + ')', xy=(0.95, 0.8), xycoords='axes fraction',
                horizontalalignment='right', verticalalignment='top', fontsize=11, color='dimgrey')   
        
        #drawing the boxplots. 
        bp = plt.boxplot(genomeLengthByGroup[j], showmeans=True, sym='', widths=boxWidth, vert=False, 
                         boxprops=dict(color=colors[j]),positions=[maxHeight-0.05],
                         capprops=dict(markerfacecolor=colors[j]),medianprops=dict(color=colors[j]), 
                         meanprops=dict(markerfacecolor=colors[j]), whis='range')        
        plt.setp(bp['whiskers'], color=colors[j])
        plt.setp(bp['caps'], color=colors[j])       
        
        
        maxBarHeight = maxHeight - 0.2
        xmin = 3
        xmax = 7
        numticksy=2
        numticksx=5
        
        plt.xlim(xmin,xmax)        
        plt.ylim(0,maxHeight)
        ax1.set_yticks(np.linspace(0,maxBarHeight,numticksy))
        ax1.set_yticklabels(np.linspace(0,maxBarHeight, numticksy), color='dimgrey' )
        ax1.set_xticks(np.linspace(xmin,xmax,numticksx*2-1))
            
        #The following lines are made so that I can have more ticks but every other label is skipped.     
        labelList= np.linspace(xmin,xmax,numticksx)
        #a list of zeros as placeholders
        newLabelList = [0] * (len(labelList) * 2 - 1)
        #making a list of original tick labels interspaced with zeros. In the code, I make zero labels invisible. 
        newLabelList[0::2] = labelList
        
        ax1.set_xticklabels(newLabelList,color='dimgrey')
        plt.tick_params(axis='both', which='both', labelsize=12) 
        
        #making the zero xlabels invisible. They were just placeholders. 
        for label in ax1.set_xticklabels(newLabelList)[1::2]:
            label.set_visible(False)
         
        #this piece is written so that the x axis of all plots except the very last don't get labeled. 
        if j in counter[:-1]:
            ax1.set_xticklabels([])


        #just added to be able to perserve text when taking the plot image into illustrator version 6
        plt.rcParams['pdf.fonttype'] = 42  
        
        #writing the stats to the file we started earlier 
        #Note that the genomeLengthByGroup in this case contains log values already. So to get actual values, I had to convert from log values. 
        minimum ="%.1f" %min(np.power(10, genomeLengthByGroup[j]))
        logmin = "%.1f" %min(genomeLengthByGroup[j])
        maximum= "%.1f" %max(np.power(10, genomeLengthByGroup[j]))
        logmax = "%.1f" %max(genomeLengthByGroup[j])
        iqr1 = "%.1f" %np.percentile(np.power(10, genomeLengthByGroup[j]), 25, interpolation='lower')
        logiqr1= "%.1f" %np.percentile(genomeLengthByGroup[j], 25, interpolation='lower')
        med ="%.1f" %np.median(np.power(10,genomeLengthByGroup[j]))
        logmed = "%.1f" %np.median(genomeLengthByGroup[j])        
        iqr3 = "%.1f" %np.percentile(np.power(10, genomeLengthByGroup[j]), 75, interpolation='higher')
        logiqr3= "%.1f" %np.percentile(genomeLengthByGroup[j], 75, interpolation='higher')        
        avg ="%.1f" %np.mean(np.power(10,genomeLengthByGroup[j]))
        logavg = "%.1f" %np.mean(genomeLengthByGroup[j])
        std ="%.1f" %np.std(np.power(10,genomeLengthByGroup[j]))
        logstd = "%.1f" %np.std(genomeLengthByGroup[j])        
        fo.write(hostNames[j]+' (N = ' + str(yVec[j]) + ')'+'\t'+ minimum +'\t'+ maximum + '\t'+ iqr1 + '\t'+ med+ '\t' + iqr3 + '\t' + avg +'\t'+ std + '\t'+ logmin + '\t' + logmax +'\t' + logiqr1 +'\t'+ logmed+ '\t'+ logiqr3 + '\t' + logavg +'\t'+ logstd + '\n')


genomeLengths = []
lengthData = []
counter = 0
length_lists = []
hostTaxidList = []
taxidList = []
folders = []
dbObject = ete.NCBITaxa()

for folder in os.listdir('all_faa'):
    #these entries had problems with their annotation, OR they are satellite viruses and need to be excluded    
    if folder not in badFolders and folder not in sats:   
        length_lists.append([])
        for fileName in os.listdir(os.path.join('all_faa', folder)):
            with open(os.path.join('all_faa', folder, fileName), "r") as faa_file:
                proteins = SeqIO.parse(faa_file, 'fasta')
                for protein in proteins:
                    length_lists[counter].append(float(len(protein.seq)))

        hostTaxid = getHostTaxidFromGenomeFolder(folder)
        # viruses with unidentified hosts 
        if hostTaxid and hostTaxid != 11086 and hostTaxid != 10036 and hostTaxid != 37965 and hostTaxid != 10640  and hostTaxid != 32644 and hostTaxid != 1 and hostTaxid != 212035 and hostTaxid != 1051671:
            taxidList.append(getTaxidFromGenomeFolder(folder))
            hostTaxidList.append(hostTaxid)
            genomeLengths.append(getGenomeLengthFromGenomeFolder(folder))
            folders.append(folder)
            counter += 1
        else:
            length_lists = length_lists[:-1]
        
        #Uncomment the following lines to see the code's progress while its running
        """
        if counter % 100 == 0:
            print(str(int(np.round(float(counter) * 100.0 / 2701.0))) + '% done.')
        """
        
taxidLineages = [dbObject.get_lineage(t) for t in taxidList]
englishLineages = [[str(rank) for rank in dbObject.translate_to_names(lin)]
                   for lin in taxidLineages]
hostTaxidLineages = [dbObject.get_lineage(t) for t in hostTaxidList]
hostEnglishLineages = [[str(rank) for rank in dbObject.translate_to_names(lin)] 
                       for lin in hostTaxidLineages]

for i in range(len(length_lists)):
    lengthData.append({'length_list' : length_lists[i],
                       'taxid' : taxidList[i],
                       'hostTaxid' : hostTaxidList[i],
                       'englishLineage' : englishLineages[i],
                       'taxidLineage' : taxidLineages[i],
                       'hostEnglishLineage' : hostEnglishLineages[i],
                       'hostTaxidLineage' : hostTaxidLineages[i],
                       'mean' : np.mean(length_lists[i]),
                       'std' : np.std(length_lists[i]),
                       'median' : np.median(length_lists[i]),
                       'numProteins' : len(length_lists[i]),
                       'genomeLength' : genomeLengths[i],
                       'folder' : folders[i]})
                       
lengthData = [virus for virus in lengthData if virus['genomeLength'] != -1]
'''
for (i, virus) in enumerate(lengthData):
    print(str(i) + '/' + str(len(lengthData)))
    (totalCoding, totalOverlap) = getCodingNumberAndOverlapNumberFromGenomeFolder(virus['folder'])
    virus['percentNonCoding'] = 100 * (virus['genomeLength'] - totalCoding) / virus['genomeLength']
    virus['percentOverlap'] = 100 * totalOverlap / virus['genomeLength']
'''
plt.figure(1, figsize=(15, 9))

#Make the host-classified plot
hostNames = 'Eukaryota;Bacteria;Archaea'.split(';')
virusGroups = [[genome for genome in lengthData if taxon in genome['hostEnglishLineage']]
               for taxon in hostNames]

plotPositionVert =4
hostNames = [x + ' Viruses' for x in hostNames]
colors = ['navy']*3
makePlot(virusGroups, hostNames, plotPositionVert)

    
#Make the Baltimore-classified plot
virusGroups = []
virusGroups.append([g for g in lengthData if 'dsDNA viruses, no RNA stage' in g['englishLineage']])
virusGroups.append([g for g in lengthData if 'ssDNA viruses' in g['englishLineage']])
virusGroups.append([g for g in lengthData if 'dsRNA viruses' in g['englishLineage']])
virusGroups.append([g for g in lengthData if 'ssRNA positive-strand viruses, no DNA stage'
                 in g['englishLineage']])
virusGroups.append([g for g in lengthData if 'ssRNA negative-strand viruses' in g['englishLineage']])
virusGroups.append([g for g in lengthData if 'Metaviridae' in g['englishLineage']
                    or 'Retroviridae' in g['englishLineage']])
virusGroups.append([g for g in lengthData if 'Caulimoviridae' in g['englishLineage']
                    or 'Hepadnaviridae' in g['englishLineage']])

hostNames = ['Group I (dsDNA)', 'Group II (ssDNA)', 'Group III (dsRNA)', 'Group IV (+ssRNA)',
             'Group V (-ssRNA)', 'Group VI (ssRNA-RT)', 'Group VII (dsDNA-RT)', 'Unclassified']
plotPositionVert =1
colors = ["navy"]*7
makePlot(virusGroups, hostNames,plotPositionVert)

    
#Make the Nucleic-Acid classified plot
newVirusGroups = []
newVirusGroups.append(virusGroups[0] + virusGroups[1] + virusGroups[6])
newVirusGroups.append(virusGroups[2] + virusGroups[3] + virusGroups[4] + virusGroups[5])
DNAviruses = newVirusGroups[0]
RNAviruses = newVirusGroups[1]
plotPositionVert = 2
colors = ["navy"]*2
makePlot(newVirusGroups, ['DNA Viruses', 'RNA Viruses'],plotPositionVert)


#Make the ICTV classified plot
hostNames = ['Caudovirales', 'Herpesvirales', 'Ligamenvirales', 'Mononegavirales', 'Nidovirales',
             'Picornavirales', 'Tymovirales']
virusGroups = [[genome for genome in lengthData if taxon in genome['englishLineage']]
               for taxon in hostNames]
colors = ["navy"]*7
plotPositionVert =3
makePlot(virusGroups, hostNames, plotPositionVert)

plt.savefig(filename + '.pdf')

#closing the text file
fo.close()
plt.show()


###A function for plotting viral sub-groups

In [2]:
#numcol and numrow refer to the number of rows and columns to be generated for subplots. Colors refers to the color used for each subplot.
#allViruses is a list of lists, containing genome lengths of different viral groups
def histPlots(allViruses, hostNames, colors, numcol, numrow):
    
    input_var = input("Enter file name: ")
    inputFileName = input_var + '.pdf'
    
    #put in place to match these plot sizes to those made in the main code
    plt.figure(1, figsize=(15, 9))
    
    #boxWidth is the width of the box plot
    boxWidth = 0.05 
    matWeight=[]
       
    counter = range(len(allViruses))
    #per each classification system, we're looping through viral groups, which correspond to j. 
    for j in counter:
        #a vector of ones the length of allViruses[j] which is a genome length vector of one of the three domains of life, depending on j. 
        vecOfOnes= np.zeros(len(allViruses[j])) + 1.
        
        #matOfOnes.append(vecOfOnes)
        matWeight.append([k / len(allViruses[j]) for k in vecOfOnes]) 
        
        #need 3 rows, 1 column for this current plot
        ax1 = plt.subplot(numcol,numrow, j+1)
        
        #calculating the number of bins needed according to Freedman-Diaconis rule. Bin width = 2 * IQR * N^(-1/3), where N is number of datapoints, IQR is interquartile range. 
        #number of bins can be calculated from bin width according to bin number = (max(data) - min(data))/bin width  
        iqr=np.percentile(allViruses[j], 75, interpolation='higher') - np.percentile(allViruses[j], 25, interpolation='lower')
        bottom = binWidth=2*iqr*len(allViruses[j])**(-1/3)
        top = max(allViruses[j])-min(allViruses[j])
        binNumFreed =int(np.ceil(top/bottom))
        
            
        #plotting the normalized histograms, and ensuring bins of equal length by setting 'bins' equal to an array with equal spacings
        n, bins, patches = plt.hist(allViruses[j], bins=binNumFreed, weights=matWeight[j], color=colors[j], linewidth=0)
        maxHeight= np.round(np.amax(n),1) + 0.2
        

        ax1.annotate(hostNames[j] + ' (N = ' + str(len(allViruses[j])) + ')', xy=(0.95, 0.8), xycoords='axes fraction',
                horizontalalignment='right', verticalalignment='top', fontsize=11, color='dimgrey')   

        #drawing the boxplots. 
        bp = plt.boxplot(allViruses[j], showmeans=True, sym='', widths=boxWidth, vert=False, 
                            boxprops=dict(color=colors[j]),positions=[maxHeight-0.05],
                            capprops=dict(markerfacecolor=colors[j]),medianprops=dict(color=colors[j]), 
                            meanprops=dict(markerfacecolor=colors[j]), whis='range')        
        plt.setp(bp['whiskers'], color=colors[j])
        plt.setp(bp['caps'], color=colors[j])         
        maxBarHeight = maxHeight - 0.2
       
        xmin = 3
        xmax = 5
        #xmax = 7
        numticksy=2
        numticksx=5
       
        plt.xlim(xmin,xmax)        
        plt.ylim(0,maxHeight)
        ax1.set_yticks(np.linspace(0,maxBarHeight,numticksy))
        ax1.set_yticklabels(np.linspace(0,maxBarHeight, numticksy), color='dimgrey' )
        ax1.set_xticks(np.linspace(xmin,xmax,numticksx*2-1))
            
        #The following lines are made so that I can have more ticks but everyother label is skipped.     
        labelList= np.linspace(xmin,xmax,numticksx)
        #a list of zeros as placeholders
        newLabelList = [0] * (len(labelList) * 2 - 1)
        #making a list of original tick labels interspaced with zeros. In the code, I make zero labels invisible. 
        newLabelList[0::2] = labelList
        
        ax1.set_xticklabels(newLabelList,color='dimgrey')
        plt.tick_params(axis='both', which='both', labelsize=12) 
        
        #making the zero xlabels invisible. They were just placeholders. 
        for label in ax1.set_xticklabels(newLabelList)[1::2]:
            label.set_visible(False)        
        
        
        #this piece is written so that the x axis of all plots except the very last don't get labeled. 
        if j in counter[:-1]:
            ax1.set_xticklabels([])

                
        #just added to be able to perserve text when taking the plot image into illustrator version 6    
        plt.rcParams['pdf.fonttype'] = 42 

    plt.savefig(inputFileName)
    plt.show()
    
    

###All viruses with known hosts
Log10 genome lengths (x axis), Normalized frequency (y axis)

In [3]:
virusesE = [np.log10(virus['genomeLength']) for virus in lengthData]
allViruses = [virusesE]
hostNames= ['All Viruses']
colors=['dimgray']
numrow = 7
numcol = 3
histPlots(allViruses, hostNames, colors, numrow, numcol)

Enter file name: allVirusesGenomeLength


###dsDNA viruses of eukaryotes
Log10 genome lengths (x axis), Normalized frequency (y axis)

In [4]:
virusesdsDNAEUKS = [np.log10(virus['genomeLength']) for virus in lengthData if 'dsDNA viruses, no RNA stage' in virus['englishLineage'] and 'Eukaryota' in virus['hostEnglishLineage']]
virusesBacu = [np.log10(virus['genomeLength']) for virus in lengthData if 'Baculoviridae' in virus['englishLineage']]
virusesPoxv = [np.log10(virus['genomeLength']) for virus in lengthData if 'Poxviridae' in virus['englishLineage']]
virusesHerp = [np.log10(virus['genomeLength']) for virus in lengthData if 'Herpesvirales' in virus['englishLineage']]
virusesPapi = [np.log10(virus['genomeLength']) for virus in lengthData if 'Papillomaviridae' in virus['englishLineage']]
virusesAden = [np.log10(virus['genomeLength']) for virus in lengthData if 'Adenoviridae' in virus['englishLineage']]
virusesPoly = [np.log10(virus['genomeLength']) for virus in lengthData if 'Polyomaviridae' in virus['englishLineage']]

allViruses= [virusesdsDNAEUKS, virusesBacu,virusesPoxv,virusesHerp,virusesPapi,virusesAden,virusesPoly]
hostNames= ['All Eukaryotic dsDNA viruses', 'Baculoviridae', 'Poxviridae', 'Herpesvirales', 'Papillomaviridae', 'Adenoviridae', 'Polyomaviridae']
colors=['lightcoral']*7
numrow = 7
numcol = 3

histPlots(allViruses, hostNames, colors, numrow, numcol)

input_var = input("Enter file name: ")
filename = input_var+ '.txt'
fo = open(filename, 'w')
#writing the first line here, and the rest as part of the for loop 
fo.write('\t'+'Genome Length Statistics For Different Viral Groups (in bases)'+'\n')
fo.write('\t'+ 'Min' +'\t'+ 'Max' + '\t'+ '25th Percentile'+'\t'+ 'Median'+ '\t' + '75th Percentile' +'\t' + 'Mean' + '\t' +'Stdev.' +'\n')

for j in range(len(allViruses)):  
    #writing the stats to the file we started earlier 
    minimum ="%.1f" %min(np.power(10, allViruses[j]))
    maximum= "%.1f" %max(np.power(10, allViruses[j]))
    iqr1 = "%.1f" %np.percentile(np.power(10, allViruses[j]), 25, interpolation='lower')
    med ="%.1f" %np.median(np.power(10,allViruses[j]))
    iqr3 = "%.1f" %np.percentile(np.power(10, allViruses[j]), 75, interpolation='higher')
    avg ="%.1f" %np.mean(np.power(10,allViruses[j]))
    std ="%.1f" %np.std(np.power(10,allViruses[j]))
    
    fo.write(hostNames[j]+' (N = ' + str(len(allViruses[j])) + ')' +'\t'+ minimum +'\t'+ maximum + '\t'+ iqr1 + '\t'+ med+ '\t' + iqr3 + '\t' + avg +'\t'+ std + '\n')

fo.close()

Enter file name: eukdsDNAgenomelength
Enter file name: eukdsDNAgenomelength


###dsDNA viruses of bacteria
Log10 genome lengths (x axis), Normalized frequency (y axis)

In [6]:
virusesdsDNABACT = [np.log10(virus['genomeLength']) for virus in lengthData if 'dsDNA viruses, no RNA stage' in virus['englishLineage'] and 'Bacteria' in virus['hostEnglishLineage']]
virusesCaudo = [np.log10(virus['genomeLength']) for virus in lengthData if 'Caudovirales' in virus['englishLineage'] and 'Bacteria' in virus['hostEnglishLineage']]
virusesSiph = [np.log10(virus['genomeLength']) for virus in lengthData if 'Caudovirales' in virus['englishLineage'] and 'Siphoviridae' in virus['englishLineage']]
virusesPodo = [np.log10(virus['genomeLength']) for virus in lengthData if 'Caudovirales' in virus['englishLineage'] and 'Podoviridae' in virus['englishLineage']]
virusesMyov = [np.log10(virus['genomeLength']) for virus in lengthData if 'Caudovirales' in virus['englishLineage'] and 'Myoviridae' in virus['englishLineage']]
virusesCaudoUncl = [np.log10(virus['genomeLength']) for virus in lengthData if 'Caudovirales' in virus['englishLineage'] and 'unclassified Caudovirales' in virus['englishLineage']]
virusesUncl = [np.log10(virus['genomeLength']) for virus in lengthData if 'unclassified dsDNA phages' in virus['englishLineage'] and 'Bacteria' in virus['hostEnglishLineage']]


allViruses= [virusesdsDNABACT, virusesCaudo,virusesSiph,virusesPodo,virusesMyov, virusesCaudoUncl, virusesUncl]

hostNames= ['All Bacterial dsDNA viruses', 'Bacterial Caudovirales', 'Bacterial Caudovirales-Siphoviridae', 'Bacterial Caudovirales-Podoviridae', 'Bacterial Caudovirales-Myoviridae', 'Bacterial Caudovirales-Unclassified','Unclassified Bacterial Phages']
colors=['cornflowerblue']*7
numrow = 7
numcol = 3

histPlots(allViruses, hostNames, colors, numrow, numcol)

input_var = input("Enter file name: ")
filename = input_var+ '.txt'
fo = open(filename, 'w')
#writing the first line here, and the rest as part of the for loop 
fo.write('\t'+'Genome Length Statistics For Different Viral Groups (in bases)'+'\n')
fo.write('\t'+ 'Min' +'\t'+ 'Max' + '\t'+ '25th Percentile'+'\t'+ 'Median'+ '\t' + '75th Percentile' +'\t' + 'Mean' + '\t' +'Stdev.' +'\n')

for j in range(len(allViruses)):  
    #writing the stats to the file we started earlier 
    minimum ="%.1f" %min(np.power(10, allViruses[j]))
    maximum= "%.1f" %max(np.power(10, allViruses[j]))
    iqr1 = "%.1f" %np.percentile(np.power(10, allViruses[j]), 25, interpolation='lower')
    med ="%.1f" %np.median(np.power(10,allViruses[j]))
    iqr3 = "%.1f" %np.percentile(np.power(10, allViruses[j]), 75, interpolation='higher')
    avg ="%.1f" %np.mean(np.power(10,allViruses[j]))
    std ="%.1f" %np.std(np.power(10,allViruses[j]))
    
    fo.write(hostNames[j]+' (N = ' + str(len(allViruses[j])) + ')' +'\t'+ minimum +'\t'+ maximum + '\t'+ iqr1 + '\t'+ med+ '\t' + iqr3 + '\t' + avg +'\t'+ std + '\n')

fo.close()

Enter file name: bacdsDNAgenomelength
Enter file name: bacdsDNAgenomelength


###dsDNA viruses of archaea
Log10 genome lengths (x axis), Normalized frequency (y axis)

In [7]:
virusesdsDNAArch = [np.log10(virus['genomeLength']) for virus in lengthData if 'dsDNA viruses, no RNA stage' in virus['englishLineage'] and 'Archaea' in virus['hostEnglishLineage']]
virusesCaudo = [np.log10(virus['genomeLength']) for virus in lengthData if 'Caudovirales' in virus['englishLineage'] and 'Archaea' in virus['hostEnglishLineage']]
virusesLiga = [np.log10(virus['genomeLength']) for virus in lengthData if 'Ligamenvirales' in virus['englishLineage'] and 'Archaea' in virus['hostEnglishLineage']]
virusesFuse = [np.log10(virus['genomeLength']) for virus in lengthData if 'Fuselloviridae' in virus['englishLineage'] and 'Archaea' in virus['hostEnglishLineage']]
virusesUncl = [np.log10(virus['genomeLength']) for virus in lengthData if 'unclassified dsDNA viruses' in virus['englishLineage'] and 'Archaea' in virus['hostEnglishLineage']]

allViruses= [virusesdsDNAArch, virusesCaudo,virusesLiga, virusesFuse, virusesUncl]

hostNames= ['All Archaeal dsDNA viruses', 'Archaeal Caudovirales', 'Ligamenvirales', 'Fuselloviridae', 'Archaeal dsDNA viruses- unclassified']
colors=['teal']*5
numrow = 7
numcol = 3

histPlots(allViruses, hostNames, colors, numrow, numcol)

input_var = input("Enter file name: ")
filename = input_var+ '.txt'
fo = open(filename, 'w')
#writing the first line here, and the rest as part of the for loop 
fo.write('\t'+'Genome Length Statistics For Different Viral Groups (in bases)'+'\n')
fo.write('\t'+ 'Min' +'\t'+ 'Max' + '\t'+ '25th Percentile'+'\t'+ 'Median'+ '\t' + '75th Percentile' +'\t' + 'Mean' + '\t' +'Stdev.' +'\n')

for j in range(len(allViruses)):  
    #writing the stats to the file we started earlier 
    minimum ="%.1f" %min(np.power(10, allViruses[j]))
    maximum= "%.1f" %max(np.power(10, allViruses[j]))
    iqr1 = "%.1f" %np.percentile(np.power(10, allViruses[j]), 25, interpolation='lower')
    med ="%.1f" %np.median(np.power(10,allViruses[j]))
    iqr3 = "%.1f" %np.percentile(np.power(10, allViruses[j]), 75, interpolation='higher')
    avg ="%.1f" %np.mean(np.power(10,allViruses[j]))
    std ="%.1f" %np.std(np.power(10,allViruses[j]))
    
    fo.write(hostNames[j]+' (N = ' + str(len(allViruses[j])) + ')' +'\t'+ minimum +'\t'+ maximum + '\t'+ iqr1 + '\t'+ med+ '\t' + iqr3 + '\t' + avg +'\t'+ std + '\n')

fo.close()

Enter file name: archdsDNAgenomelength
Enter file name: archdsDNAgenomelength


###ssDNA viruses of eukaryotes
Log10 genome lengths (x axis), Normalized frequency (y axis)

In [8]:
virusesssDNAEUKS = [np.log10(virus['genomeLength']) for virus in lengthData if 'ssDNA viruses' in virus['englishLineage'] and 'Eukaryota' in virus['hostEnglishLineage']]
virusesGemi = [np.log10(virus['genomeLength']) for virus in lengthData if 'Geminiviridae' in virus['englishLineage'] and 'Eukaryota' in virus['hostEnglishLineage']]
virusesCirc = [np.log10(virus['genomeLength']) for virus in lengthData if 'Circoviridae' in virus['englishLineage'] and 'Eukaryota' in virus['hostEnglishLineage']]
virusesParv = [np.log10(virus['genomeLength']) for virus in lengthData if 'Parvoviridae' in virus['englishLineage'] and 'Eukaryota' in virus['hostEnglishLineage']]
virusesAnel = [np.log10(virus['genomeLength']) for virus in lengthData if 'Anelloviridae' in virus['englishLineage'] and 'Eukaryota' in virus['hostEnglishLineage']]
virusesNano = [np.log10(virus['genomeLength']) for virus in lengthData if 'Nanoviridae' in virus['englishLineage'] and 'Eukaryota' in virus['hostEnglishLineage']]
virusesUncl = [np.log10(virus['genomeLength']) for virus in lengthData if 'unclassified ssDNA viruses' in virus['englishLineage'] and 'Eukaryota' in virus['hostEnglishLineage']]

allViruses= [virusesssDNAEUKS, virusesGemi,virusesCirc,virusesParv,virusesAnel,virusesNano,virusesUncl]
hostNames= ['All Eukaryotic ssDNA viruses', 'Geminiviridae', 'Circoviridae', 'Parvoviridae', 'Anelloviridae', 'Nanoviridae', 'Unclassified Eukaryotic ssDNA viruses']
colors=['lightcoral', 'lightcoral','lightcoral','lightcoral','lightcoral','lightcoral','lightcoral']
numrow = 7
numcol = 3
histPlots(allViruses, hostNames, colors, numrow, numcol)


input_var = input("Enter file name: ")
filename = input_var+ '.txt'
fo = open(filename, 'w')
#writing the first line here, and the rest as part of the for loop 
fo.write('\t'+'Genome Length Statistics For Different Viral Groups (in bases)'+'\n')
fo.write('\t'+ 'Min' +'\t'+ 'Max' + '\t'+ '25th Percentile'+'\t'+ 'Median'+ '\t' + '75th Percentile' +'\t' + 'Mean' + '\t' +'Stdev.' +'\n')

for j in range(len(allViruses)):  
    #writing the stats to the file we started earlier 
    minimum ="%.1f" %min(np.power(10, allViruses[j]))
    maximum= "%.1f" %max(np.power(10, allViruses[j]))
    iqr1 = "%.1f" %np.percentile(np.power(10, allViruses[j]), 25, interpolation='lower')
    med ="%.1f" %np.median(np.power(10,allViruses[j]))
    iqr3 = "%.1f" %np.percentile(np.power(10, allViruses[j]), 75, interpolation='higher')
    avg ="%.1f" %np.mean(np.power(10,allViruses[j]))
    std ="%.1f" %np.std(np.power(10,allViruses[j]))
    
    fo.write(hostNames[j]+' (N = ' + str(len(allViruses[j])) + ')' +'\t'+ minimum +'\t'+ maximum + '\t'+ iqr1 + '\t'+ med+ '\t' + iqr3 + '\t' + avg +'\t'+ std + '\n')

fo.close()

Enter file name: eukssDNAgenomelength
Enter file name: eukssDNAgenomelength


###ssDNA viruses of Bacteria
Log10 genome lengths (x axis), Normalized frequency (y axis)

In [9]:
virusesssDNABact = [np.log10(virus['genomeLength']) for virus in lengthData if 'ssDNA viruses' in virus['englishLineage'] and 'Bacteria' in virus['hostEnglishLineage']]
virusesMicr = [np.log10(virus['genomeLength']) for virus in lengthData if 'Microviridae' in virus['englishLineage'] and 'Bacteria' in virus['hostEnglishLineage']]
virusesInov = [np.log10(virus['genomeLength']) for virus in lengthData if 'Inoviridae' in virus['englishLineage'] and 'Bacteria' in virus['hostEnglishLineage']]

allViruses= [virusesssDNABact, virusesMicr,virusesInov]
hostNames= ['All Bacterial ssDNA viruses', 'Microviridae', 'Inoviridae']
colors=['cornflowerblue', 'cornflowerblue','cornflowerblue']
numrow = 7
numcol = 3
histPlots(allViruses, hostNames, colors, numrow, numcol)

input_var = input("Enter file name: ")
filename = input_var+ '.txt'
fo = open(filename, 'w')
#writing the first line here, and the rest as part of the for loop 
fo.write('\t'+'Genome Length Statistics For Different Viral Groups (in bases)'+'\n')
fo.write('\t'+ 'Min' +'\t'+ 'Max' + '\t'+ '25th Percentile'+'\t'+ 'Median'+ '\t' + '75th Percentile' +'\t' + 'Mean' + '\t' +'Stdev.' +'\n')

for j in range(len(allViruses)):  
    #writing the stats to the file we started earlier 
    minimum ="%.1f" %min(np.power(10, allViruses[j]))
    maximum= "%.1f" %max(np.power(10, allViruses[j]))
    iqr1 = "%.1f" %np.percentile(np.power(10, allViruses[j]), 25, interpolation='lower')
    med ="%.1f" %np.median(np.power(10,allViruses[j]))
    iqr3 = "%.1f" %np.percentile(np.power(10, allViruses[j]), 75, interpolation='higher')
    avg ="%.1f" %np.mean(np.power(10,allViruses[j]))
    std ="%.1f" %np.std(np.power(10,allViruses[j]))
    
    fo.write(hostNames[j]+' (N = ' + str(len(allViruses[j])) + ')' +'\t'+ minimum +'\t'+ maximum + '\t'+ iqr1 + '\t'+ med+ '\t' + iqr3 + '\t' + avg +'\t'+ std + '\n')

fo.close()

Enter file name: bacssDNAgenomelength
Enter file name: bacssDNAgenomelength


###A plot function for exploring linear regressions

In [24]:
from decimal import Decimal
def plotmaker(x,y,xlab,ylab, xmin, xmax, ymin, ymax):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x,y, color ='navy', alpha=0.5)
    ax.set_xlabel(xlab, fontsize=14, color='dimgrey')
    ax.set_ylabel(ylab, fontsize=14, color='dimgrey' )
    ax.tick_params(axis='both', which='both', colors='dimgrey',labelsize=12) 

    #just added to be able to perserve text when taking the plot image into illustrator
    plt.rcParams['pdf.fonttype'] = 42  
    plt.xlim(xmin,xmax)
    plt.ylim(ymin,ymax)
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    ax.plot(x, np.poly1d(np.polyfit(x, y, 1))(x), color = 'dimgrey', alpha=0.5)
    text1 = 'y = ' + str(round(slope,4)) + 'x' ' + ' + str(int((round(intercept))))
    p_value2 = '%.1E' % Decimal(p_value)
    text2 = 'r = '+ str(round(r_value,2)) + ', p = '+ p_value2
    text3= 'N = '+str(len(x))
    ax.text(.25, .9,text1,horizontalalignment='center',verticalalignment='center',transform = ax.transAxes, fontsize=14, color='dimgrey')
    ax.text(.25, .8,text2,horizontalalignment='center',verticalalignment='center',transform = ax.transAxes, fontsize=14, color='dimgrey')
    ax.text(.25, .7,text3,horizontalalignment='center',verticalalignment='center',transform = ax.transAxes, fontsize=14, color='dimgrey')
    return()

###Number of genes versus genome length for all dsDNA viruses


In [3]:
xB = [virus['genomeLength'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
xE= [virus['genomeLength'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
xA= [virus['genomeLength'] for virus in lengthData if 'Archaea' in virus['hostEnglishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
x= xB + xE + xA

colB=['cornflowerblue']*len(xB)
colE=['salmon']*len(xE)
colA=['teal']*len(xA)

col=colB+colE+colA

yB= [virus['numProteins'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
yE=[virus['numProteins'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage']  and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
yA=[virus['numProteins'] for virus in lengthData if 'Archaea' in virus['hostEnglishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
y= yB+yE+yA

x= np.array(x)
y=np.array(y)

g = sns.jointplot(x,y, size=7, color='k', kind ='reg' )
#Clear the axes containing the scatter plot
g.ax_joint.cla()
plt.sca(g.ax_joint)
plt.xlim([0,700000])
plt.ylim([0,700])

#Plot each individual point separately
for i,row in enumerate(x):
    g.ax_joint.scatter(x[i], y[i], color=col[i], alpha=0.8)

g.set_axis_labels('Genome Length (bp)', 'Number of Genes', fontsize=16, color='gray')
g.savefig('genomeLengthVsNumGenes.pdf')
    
l = sns.jointplot(x,y, size=7, color= 'dimgrey', kind ='reg', xlim=[0,700000], ylim=[0,700])
l.set_axis_labels('Genome Length (bp)', 'Number of Genes', fontsize=16, color='gray')
l.savefig('genomeLengthVsNumGenes.pdf')


/Users/Gita/anaconda/lib/python3.4/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/Users/Gita/anaconda/lib/python3.4/site-packages/statsmodels/nonparametric/kdetools.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y = X[:m/2+1] + np.r_[0,X[m/2+1:],0]*1j


###Number of genes versus genome length for dsDNA viruses of bacteria


In [22]:
xall = [virus['genomeLength'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
xPodo= [virus['genomeLength'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and 'Podoviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
xMyo= [virus['genomeLength'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and 'Myoviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
xSipho= [virus['genomeLength'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and 'Siphoviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
majorCats= ['Siphoviridae', 'Myoviridae', 'Podoviridae'] 
xOther= [virus['genomeLength'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and not any(s in virus['englishLineage'] for s in majorCats) and "dsDNA viruses, no RNA stage" in virus['englishLineage']]         
x= xPodo + xMyo + xSipho + xOther

colPodo=['orangered']*len(xPodo)
colMyo=['y']*len(xMyo)
colSipho=['skyblue']*len(xSipho)
colOther=['grey']*len(xOther)
col=colPodo+colMyo+colSipho+colOther

yPodo= [virus['numProteins'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and 'Podoviridae' in virus['englishLineage']]
yMyo=[virus['numProteins'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and 'Myoviridae' in virus['englishLineage']]
ySipho=[virus['numProteins'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and 'Siphoviridae' in virus['englishLineage']]
yOther= [virus['numProteins'] for virus in lengthData if 'Bacteria' in virus['hostEnglishLineage'] and not any(s in virus['englishLineage'] for s in majorCats) and "dsDNA viruses, no RNA stage" in virus['englishLineage']]         

y= yPodo+yMyo+ySipho+yOther

x= np.array(x)
y=np.array(y)

g = sns.jointplot(x,y, size=7, color='k', kind ='reg' )

#Clear the axes containing the scatter plot
g.ax_joint.cla()
plt.sca(g.ax_joint)
plt.xlim([0,250000])
plt.ylim([0,500])

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
text1 = 'y = ' + str(round(slope,4)) + 'x' ' + ' + str(int((round(intercept))))
p_value2 = '%.1E' % Decimal(p_value)
text2 = 'r = '+ str(round(r_value,2)) + ', p = '+ p_value2
text3= 'N = '+str(len(x))
plt.text(10000, 450,text1,horizontalalignment='left',verticalalignment='center', fontsize=14, color='dimgrey')
plt.text(10000, 425,text2,horizontalalignment='left',verticalalignment='center', fontsize=14, color='dimgrey')
plt.text(10000, 400,text3,horizontalalignment='left',verticalalignment='center', fontsize=14, color='dimgrey')


#Plot each individual point separately
for i,row in enumerate(x):
    g.ax_joint.scatter(x[i], y[i], color=col[i], alpha=0.8)

g.set_axis_labels('Genome Length', 'Number of genes', fontsize=16, color='gray')
g.savefig('genomeLengthVsNumberofGenesBacterialdsDNA.pdf')
    
l = sns.jointplot(x,y, size=7, color= 'dimgrey', kind ='reg', xlim=[0,250000], ylim=[0,500])
l.set_axis_labels('Genome Length', 'Number of genes', fontsize=16, color='gray')

/Users/Gita/anaconda/lib/python3.4/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/Users/Gita/anaconda/lib/python3.4/site-packages/statsmodels/nonparametric/kdetools.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y = X[:m/2+1] + np.r_[0,X[m/2+1:],0]*1j


###Number of genes versus genome length for dsDNA viruses of eukaryotes


In [21]:
xall = [virus['genomeLength'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
#print(len(xall))
xBac= [virus['genomeLength'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Baculoviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
xPox= [virus['genomeLength'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Poxviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
xHerp= [virus['genomeLength'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Herpesvirales' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
xPap= [virus['genomeLength'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Papillomaviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
xAd= [virus['genomeLength'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Adenoviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
xPoly= [virus['genomeLength'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Polyomaviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]

majorCats= ['Baculoviridae', 'Poxviridae', 'Herpesvirales', 'Papillomaviridae', 'Adenoviridae','Polyomaviridae'] 

xOther= [virus['genomeLength'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and not any(s in virus['englishLineage'] for s in majorCats) and "dsDNA viruses, no RNA stage" in virus['englishLineage']]         
x= xBac+xPox+ xHerp+ xPap+ xAd+ xPoly+xOther
x2= xBac+xPox+ xHerp+ xPap+ xAd+ xPoly
#print(len(x2))
#print(len(xOther))

colBac=['goldenrod']*len(xBac)
colPox=['teal']*len(xPox)
colHerp=['slateblue']*len(xHerp)
colPap=['hotpink']*len(xPap)
colAd=['maroon']*len(xAd)
colPoly=['palegreen']*len(xPoly)
colOther=['grey']*len(xOther)
col=colBac+colPox+ colHerp+ colPap+ colAd+ colPoly+ colOther

yBac= [virus['numProteins'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Baculoviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
yPox= [virus['numProteins'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Poxviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
yHerp= [virus['numProteins'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Herpesvirales' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
yPap= [virus['numProteins'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Papillomaviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
yAd= [virus['numProteins'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Adenoviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
yPoly= [virus['numProteins'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and 'Polyomaviridae' in virus['englishLineage'] and "dsDNA viruses, no RNA stage" in virus['englishLineage']]
yOther= [virus['numProteins'] for virus in lengthData if 'Eukaryota' in virus['hostEnglishLineage'] and not any(s in virus['englishLineage'] for s in majorCats) and "dsDNA viruses, no RNA stage" in virus['englishLineage']]         

y= yBac+yPox+ yHerp+ yPap+ yAd+ yPoly+yOther

x= np.array(x)
y=np.array(y)

g = sns.jointplot(x,y, size=7, color='k', kind ='reg' )

#Clear the axes containing the scatter plot
g.ax_joint.cla()
plt.sca(g.ax_joint)
plt.xlim([0,500000])
plt.ylim([0,500])

slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
text1 = 'y = ' + str(round(slope,4)) + 'x' ' + ' + str(int((round(intercept))))
p_value2 = '%.1E' % Decimal(p_value)
text2 = 'r = '+ str(round(r_value,2)) + ', p = '+ p_value2
text3= 'N = '+str(len(x))
plt.text(10000, 450,text1,horizontalalignment='left',verticalalignment='center', fontsize=14, color='dimgrey')
plt.text(10000, 425,text2,horizontalalignment='left',verticalalignment='center', fontsize=14, color='dimgrey')
plt.text(10000, 400,text3,horizontalalignment='left',verticalalignment='center', fontsize=14, color='dimgrey')


#Plot each individual point separately
for i,row in enumerate(x):
    g.ax_joint.scatter(x[i], y[i], color=col[i], alpha=0.4)

g.set_axis_labels('Genome Length', 'Number of Genes', fontsize=16, color='gray')
g.savefig('genomeLengthVsNumGenesEukdsDNA.pdf')
    
l = sns.jointplot(x,y, size=7, color= 'dimgrey', kind ='reg', xlim=[0,700000], ylim=[0,600])
l.set_axis_labels('Genome Length', 'Number of Genes', fontsize=16, color='gray')
l.savefig('genomeLengthVsNumGenesEukdsDNA.pdf')


/Users/Gita/anaconda/lib/python3.4/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/Users/Gita/anaconda/lib/python3.4/site-packages/statsmodels/nonparametric/kdetools.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y = X[:m/2+1] + np.r_[0,X[m/2+1:],0]*1j


###Number of genes versus genome length for dsDNA viruses of archaea


In [25]:
x = [virus['genomeLength'] for virus in lengthData if 'dsDNA viruses, no RNA stage' in virus['englishLineage'] and 'Archaea' in virus['hostEnglishLineage']]
y= [virus['numProteins'] for virus in lengthData if 'dsDNA viruses, no RNA stage' in virus['englishLineage'] and 'Archaea' in virus['hostEnglishLineage']]
plotmaker(x,y,'genome length (bp)', 'number of genes', 0, 100000, 0, 200)

()